In [10]:
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
def renameColumns(df):
    return [col.split('-', 1)[1] if '-' in col else col for col in df.columns]

def convertTime(df):
    if df["Time"].dtype == 'int64':
        df["Time"] = pd.to_datetime(df["Time"], unit='ms')    
    else:
        df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')
    df.sort_values(by=['Time'], inplace=True)
    return df

In [12]:
def readAndProcessCSV(filename):
    df = pd.read_csv(filename)
    df.columns = renameColumns(df)
    df = convertTime(df)
    return df

In [13]:
def applyMask(df, mask):
    dfWeekend = df[~mask]
    dfWeekend = dfWeekend.reset_index(drop=True)
    return dfWeekend

def eliminateNightHours(df):
    return df[(df['Time'].dt.hour >= 6) & (df['Time'].dt.hour <= 23)]

def isWeekend(dateObj):
    return dateObj.weekday() in [5, 6]

def isWeeekday(dateObj):
    return not isWeekend(dateObj)

In [14]:
def makeDfSensorSingle(df, sensorName):
    sensorColumns = ['Time'] + [col for col in df.columns if sensorName in col] 
    dfSensor = df[sensorColumns]
    dfSensor = dfSensor.dropna(axis=1, how='all')
    dfSensor = dfSensor.dropna().reset_index(drop=True)
    return dfSensor

#drop all columns that contain the word 'vdd'
def dropVddColumns(df):
    return df[df.columns.drop(list(df.filter(regex='vdd')))]

In [15]:
#DF that has all the data from October and November
dfOctNov = readAndProcessCSV("sensorData/08_45_18.csv")
fileNames = ["08_46_27", "08_47_03", "08_48_14", "08_48_51", "08_49_32", "08_50_13", "08_50_46"]
for fileName in fileNames:
    df = readAndProcessCSV("sensorData/" + fileName + ".csv")
    df.replace('undefined', pd.NA, inplace=True)
    dfOctNov = pd.concat([dfOctNov, df])

#Time range of dfOctNov is 2023-10-01 to 2023-11-25
#dfOctNov["Time"].dt.date

In [16]:
weekendMask = dfOctNov['Time'].apply(isWeekend)
dfFinal = applyMask(dfOctNov, weekendMask)
dfFinal = eliminateNightHours(dfFinal)
dfFinal = dropVddColumns(dfFinal)
dfFinal = dfFinal.reset_index(drop=True)
# dfFinal.head()

In [17]:
selectedSensors = ['eye03', 'eye04', 'eye05', 'eye09', 'eye11', 'gas01', 'gas02', 'gas03', 'gas04', 'gas05', 'gas06', 'voc01', 'voc02', 'voc03', 'voc04', 'voc05']
dfConcat = makeDfSensorSingle(dfFinal, 'eye02')
for i in selectedSensors:
    sensorDf = makeDfSensorSingle(dfFinal, i)
    dfConcat = pd.concat([dfConcat, sensorDf], axis=1)

dfConcat = dfConcat.drop('Time', axis=1)
dfConcat = dfConcat.dropna().reset_index(drop=True)
# dfConcat.head

In [18]:
def toNumeric(listOfColumns):
    for i in listOfColumns:
        dfConcat[i] = pd.to_numeric(dfConcat[i])

def dropColumns(listOfColumns, df):
    df.drop(listOfColumns, axis=1, inplace=True)

def computeMeanColumn(df, list, columnName):
    df[columnName] = df[list].mean(axis=1)
    df[columnName] = df[columnName].round(2)

def computeStdColumn(df, list, columnName):
    df[columnName] = df[list].std(axis=1)
    df[columnName] = df[columnName].round(2)

def computeMedianColumn(df, list, columnName):
    df[columnName] = df[list].median(axis=1)
    df[columnName] = df[columnName].round(2)

def computeMaxColumn(df, list, columnName):
    df[columnName] = df[list].max(axis=1)
    df[columnName] = df[columnName].round(2)

def computeMinColumn(df, list, columnName):
    df[columnName] = df[list].min(axis=1)
    df[columnName] = df[columnName].round(2)

def computeInterquartileRangeColumn(df, list, columnName):
    df[columnName] = df[list].quantile(0.75, axis=1) - df[list].quantile(0.25, axis=1)
    df[columnName] = df[columnName].round(2)
def processStatisticalColumns(selectedSensors, measurementName):
    columnsList = []
    for i in selectedSensors:
        column = i + '.' + measurementName
        columnsList.append(column)
    toNumeric(columnsList)
    return columnsList
    
def applyStatisticalMethod(df, selectedSensors, measurementName, statisticalMethod):
    columnsList = processStatisticalColumns(selectedSensors, measurementName)
    df[measurementName + '.' + statisticalMethod] = df[columnsList].agg(statisticalMethod, axis=1)
    dropColumns(columnsList, df)

eyeSelectedSensors = ['eye02', 'eye03', 'eye04', 'eye05', 'eye09', 'eye11']
gasSelectedSensors = ['gas01', 'gas02', 'gas03', 'gas04', 'gas05', 'gas06']
vocSelectedSensors = ['voc01', 'voc02', 'voc03', 'voc04', 'voc05']

sensorTypes = {
    'eye': eyeSelectedSensors,
    'gas': gasSelectedSensors,
    'voc': vocSelectedSensors
}

measurements = {
    'humidity': ['mean', 'std', 'median', 'min', 'max', 'iqr'],
    'light': ['mean', 'std', 'median', 'min', 'max', 'iqr'],
    'motion': ['mean', 'std', 'median', 'min', 'max', 'iqr'],
    'temperature': ['mean', 'std', 'median', 'min', 'max', 'iqr'],
}

for sensorType, sensors in sensorTypes.items():
    for measurement, aggFuncs in measurements.items():
        for aggFunc in aggFuncs:
            columnName = f'{sensorType}.{measurement}.{aggFunc}'
            computeMeanColumn(dfConcat, processStatisticalColumns(sensors, measurement), columnName)
            computeStdColumn(dfConcat, processStatisticalColumns(sensors, measurement), columnName)
            computeMedianColumn(dfConcat, processStatisticalColumns(sensors, measurement), columnName)
            computeMaxColumn(dfConcat, processStatisticalColumns(sensors, measurement), columnName)
            computeMinColumn(dfConcat, processStatisticalColumns(sensors, measurement), columnName)
            computeInterquartileRangeColumn(dfConcat, processStatisticalColumns(sensors, measurement), columnName)
        dropColumns(processStatisticalColumns(sensors, measurement), dfConcat)

columnsListCO2 = processStatisticalColumns(gasSelectedSensors, 'co2')
computeMeanColumn(dfConcat, columnsListCO2, 'gas.co2.mean')
computeStdColumn(dfConcat, columnsListCO2, 'gas.co2.std')
computeMedianColumn(dfConcat, columnsListCO2, 'gas.co2.median')
computeMaxColumn(dfConcat, columnsListCO2, 'gas.co2.max')
computeMinColumn(dfConcat, columnsListCO2, 'gas.co2.min')
computeInterquartileRangeColumn(dfConcat, columnsListCO2, 'gas.co2.iqr')
dropColumns(columnsListCO2, dfConcat)

columnsListTVOC = processStatisticalColumns(vocSelectedSensors, 'tvoc')
computeMeanColumn(dfConcat, columnsListTVOC, 'voc.tvoc.mean')
computeStdColumn(dfConcat, columnsListTVOC, 'voc.tvoc.std')
computeMedianColumn(dfConcat, columnsListTVOC, 'voc.tvoc.median')
computeMaxColumn(dfConcat, columnsListTVOC, 'voc.tvoc.max')
computeMinColumn(dfConcat, columnsListTVOC, 'voc.tvoc.min')
computeInterquartileRangeColumn(dfConcat, columnsListTVOC, 'voc.tvoc.iqr')
dropColumns(columnsListTVOC, dfConcat)

dfConcat.columns


Index(['eye02.occupancy', 'eye03.occupancy', 'eye04.occupancy',
       'eye05.occupancy', 'eye09.occupancy', 'eye11.occupancy',
       'eye.humidity.mean', 'eye.humidity.std', 'eye.humidity.median',
       'eye.humidity.min', 'eye.humidity.max', 'eye.humidity.iqr',
       'eye.light.mean', 'eye.light.std', 'eye.light.median', 'eye.light.min',
       'eye.light.max', 'eye.light.iqr', 'eye.motion.mean', 'eye.motion.std',
       'eye.motion.median', 'eye.motion.min', 'eye.motion.max',
       'eye.motion.iqr', 'eye.temperature.mean', 'eye.temperature.std',
       'eye.temperature.median', 'eye.temperature.min', 'eye.temperature.max',
       'eye.temperature.iqr', 'gas.humidity.mean', 'gas.humidity.std',
       'gas.humidity.median', 'gas.humidity.min', 'gas.humidity.max',
       'gas.humidity.iqr', 'gas.light.mean', 'gas.light.std',
       'gas.light.median', 'gas.light.min', 'gas.light.max', 'gas.light.iqr',
       'gas.motion.mean', 'gas.motion.std', 'gas.motion.median',
       'gas.moti

In [22]:
#get all rows where either eye02, eye03, eye04, eye05, eye09, eye11 have a value of 2
dfConcat = dfConcat[(dfConcat['eye02.occupancy'] == 2) | (dfConcat['eye03.occupancy'] == 2) | (dfConcat['eye04.occupancy'] == 2) | (dfConcat['eye05.occupancy'] == 2) | (dfConcat['eye09.occupancy'] == 2) | (dfConcat['eye11.occupancy'] == 2)]
dfConcat

,eye02.occupancy,eye03.occupancy,eye04.occupancy,eye05.occupancy,eye09.occupancy,eye11.occupancy,eye.humidity.mean,eye.humidity.std,eye.humidity.median,eye.humidity.min,...,gas.co2.median,gas.co2.max,gas.co2.min,gas.co2.iqr,voc.tvoc.mean,voc.tvoc.std,voc.tvoc.median,voc.tvoc.max,voc.tvoc.min,voc.tvoc.iqr
15,1.0,0.0,2.0,0.0,0.0,0.0,1.75,1.75,1.75,1.75,...,450.5,533.0,421.0,50.25,126.6,39.64,125.0,188.0,83.0,31.0
16,0.0,0.0,2.0,0.0,0.0,0.0,1.75,1.75,1.75,1.75,...,470.0,514.0,424.0,49.75,136.8,50.39,135.0,211.0,79.0,49.0
59,1.0,0.0,0.0,0.0,0.0,2.0,3.00,3.00,3.00,3.00,...,447.0,519.0,411.0,60.50,133.8,51.47,147.0,204.0,82.0,68.0
64,0.0,0.0,0.0,0.0,0.0,2.0,2.25,2.25,2.25,2.25,...,518.0,1706.0,404.0,664.25,129.8,51.50,124.0,208.0,81.0,60.0
65,0.0,0.0,0.0,0.0,0.0,2.0,2.25,2.25,2.25,2.25,...,521.5,1841.0,402.0,750.50,135.2,48.11,129.0,206.0,77.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,2.0,1.0,2.0,0.0,0.0,1.0,7.00,7.00,7.00,7.00,...,907.0,1196.0,445.0,560.50,295.6,74.86,340.0,367.0,210.0,123.0
4097,2.0,1.0,1.0,1.0,0.0,1.0,7.75,7.75,7.75,7.75,...,918.0,1250.0,465.0,521.75,294.4,71.41,310.0,370.0,215.0,127.0
4098,2.0,1.0,1.0,1.0,0.0,2.0,8.25,8.25,8.25,8.25,...,912.0,1324.0,431.0,500.75,292.2,69.77,298.0,369.0,215.0,123.0
4099,2.0,1.0,1.0,1.0,0.0,2.0,8.75,8.75,8.75,8.75,...,881.5,1393.0,422.0,525.25,300.4,57.64,321.0,349.0,214.0,76.0
